# Lesson 2: Array-oriented programming

## Programming paradigms

<br>

Programming paradigms, such as "imperative," "object-oriented," and "functional," are general styles of programming.

<br><br>

Some programming languages were designed to enable or encourage some paradigms over others. For example, C++ enabled object-oriented programming in C.

<br><br>

In array-oriented programming, the primary data type is an array, and most functions perform one operation on all the elements of the array.

<br><br>

In [ ]:
import numpy as np

This is _imperative_ programming:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
output_data = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])

for i in range(len(input_data)):             # explicitly specifies an order of execution
    output_data[i] = input_data[i]**2        # user says what happens to each element

output_data

This is _functional_ programming:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

output_data = np.fromiter(
    map(lambda x: x**2, input_data), int     # still focused on the individual element "x"
)

output_data

And this is _array-oriented_ programming:

<br>

In [ ]:
input_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

output_data = input_data**2                  # implicit indexes, no individual elements

output_data

Programming paradigms are not sharply defined. A single language and even a single codebase can use several paradigms.

<br>

They are _styles_, and they're useful because they each bring different programming concepts into the foreground:

| Paradigm | Emphasizes |
|:-:|:-:|
| imperative/procedural | low-level algorithms |
| object-oriented | large-scale program structure |
| actor-based | temporal locality |
| literate | human instruction |
| event-driven | cause-and-effect |
| declarative | properties of desired result |
| symbolic | formula transformations |
| functional | data transformations |
| array-oriented | data distributions |

## What is array-oriented programming good for?

<br>

All of the languages/major libraries that emphasize array-oriented programming (that I know of):

<br>

<img src="../img/apl-timeline.svg" width="100%">

Almost all of them are intended as _interactive data-analysis_ environments.

(Fortran-90 is the only one that is not interactive.)

<br><br>

Almost all feature:

* interactive REPL (read-evaluate-print loop)
* concise notation
* unabashedly mathematical

### The grandfather: APL

<table><tr>
    <td width="25%"><img src="../img/apl-book.png" width="100%"></td>
    <td width="50%"><img src="../img/apl-keyboard.jpg" width="100%"></td>
</tr></table>

In [ ]:
%%html
<div style="overflow: hidden;"><iframe src="https://tryapl.org/" width="100%" height="380" scrolling="no" style="border: none;"></div>

**Quizlet:** Translate the following NumPy operations into APL.

<br>

In [ ]:
np.arange(10)

<br>

In [ ]:
np.sum(np.arange(10))

<br>

In [ ]:
np.cumsum(np.arange(10))

**Answers:**

```apl
      (⍳10) - 1
0 1 2 3 4 5 6 7 8 9

      +/(⍳10) - 1
45

      +\(⍳10) - 1
0 1 3 6 10 15 21 28 36 45
```

<br>

APL was too concise! Modern array-oriented programming is looking for the right balance.

<center>
<img src="../img/tshirt.jpg" width="20%">
</center>

### Distributions and interactivity

Array-oriented languages bring data _distributions_ to the foreground.

<br>

In [ ]:
from hist import Hist  # histogram library

<br>

Given a large dataset...

In [ ]:
dataset = np.random.normal(0, 1, 1000000)  # one MILLION data points

<br>

How are the data _distributed_?

In [ ]:
Hist.new.Reg(100, -5, 5).Double().fill(dataset)

What happens if we apply a function to _all values in the distribution_?

In [ ]:
dataset2 = dataset**2

<br>

In [ ]:
Hist.new.Reg(100, -1, 10).Double().fill(dataset2)

Can anyone guess what this distribution will look like?

In [ ]:
dataset3 = np.sin(1/dataset2)

<br>

(I can't.)

In [ ]:
Hist.new.Reg(100, -1, 1).Double().fill(dataset3)

**Human readability advantage:**

  * Mathematical expressions are concise, more convenient to type interactivity.

<br>

**Computational advantage:**

  * The right _part_ of the computation is accelerated: the loop over all values in the distribution.

## NumPy

<center>
<img src="../img/Numpy_Python_Cheat_Sheet.svg" width="90%">
</center>

NumPy's version of

```apl
      10 20 30 + 1 2 3
11 22 33
```

is

In [ ]:
np.array([10, 20, 30]) + np.array([1, 2, 3])

<br><br>

Since all scalar → scalar operators (and scalar → scalar functions in `np.*`) are promoted to elementwise operations on arrays, any [closed form](https://en.wikipedia.org/wiki/Closed-form_expression) expression duck-types between scalars and arrays.

In [ ]:
def quadratic_formula(a, b, c):
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

Compute the quadratic formula on one set of scalar values:

In [ ]:
a = 5
b = 10
c = -0.1

quadratic_formula(a, b, c)

<br>

Compute the quadratic formula on a million values in arrays:

In [ ]:
a = np.random.uniform(5, 10, 1000000)
b = np.random.uniform(10, 20, 1000000)
c = np.random.uniform(-0.1, 0.1, 1000000)

quadratic_formula(a, b, c)

### Speed and memory use

<br>

Although I'll later show you some techniques that are faster than NumPy, NumPy is already a huge step beyond pure Python.

In [ ]:
%%timeit

output = []
for a_i, b_i, c_i in zip(a, b, c):
    output.append(quadratic_formula(a_i, b_i, c_i))

<br>

In [ ]:
%%timeit

output = quadratic_formula(a, b, c)

<br>

Why?

There are essentially two kinds of languages, and Python is among the slow kind.

<center>
<img src="../img/benchmark-games-2023.svg" width="70%">
</center>

Python (and Ruby, Perl, Lua, PHP...) allow all data types to be dynamic, so that a list or a dict can contain any type of data.

<br><br>

But this means that before each operation, data types need to be checked to see if the operation is possible.

<br><br>

Over and over again, for every step of a loop over a million elements.

Memory layout of a Python list.

<br>

<center>
<img src="../img/python-list-layout.svg" width="75%">
</center>

Memory layout of a NumPy array.

<br>

<center>
<img src="../img/python-array-layout.svg" width="75%">
</center>

Dynamic types are usually the main reason why pure Python is slow, but there are others (garbage collector, virtual machine, pointer chasing...).

<br><br><br>

Most of these are addressed by a compiled library like NumPy, in which the loops over large datasets are in compiled code.

### Expressibility in NumPy

Loops over large datasets are in compiled code, but can we still express what we need to compute from the Python side?

<br>

Usually. NumPy has most of the pieces that we need:

  * elementwise operations ("ufuncs")
  * broadcasting
  * basic and advanced slicing
  * slice-assignment
  * reshaping
  * reducers
  * specialized functions

**Example:** Compute the size of the spaces between consecutive elements.

In [ ]:
array = np.array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
array

<br><br><br>

In [ ]:
array[1:] - array[:-1]

<center>
<img src="../img/flat-operation.svg" width="70%">
</center>

<center>
<img src="../img/shifted-operation.svg" width="70%">
</center>

**Quizlet:** Compute the length of the curve sampled by arrays `x` and `y`.

In [ ]:
t = np.linspace(0, 2*np.pi, 10000)
x = np.sin(3*t)
y = np.sin(4*t)

<br>

<center>
<img src="../img/length-by-segment.svg" width="55%">
</center>

<br>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y);

**Answer:**

In [ ]:
np.sum(np.sqrt((x[1:] - x[:-1])**2 + (y[1:] - y[:-1])**2))

### Slicing

Python has a very concise slicing syntax:

```python
container[start:stop:step]
```

any one of which can be left out, to get a default. Negative values count backward, from the end.

In [ ]:
container = [0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]

In [ ]:
container[2:]

<br>

In [ ]:
container[:5]

<br>

In [ ]:
container[-6:-2]

<br>

In [ ]:
container[4:10:2]

<br>

In [ ]:
container[::3]

NumPy goes beyond ordinary slicing by allowing slices in multiple dimensions.

<center>
<img src="../img/numpy-slicing.png" width="25%">
</center>

In [ ]:
arr = np.array([[1.1, 2.2, 3.3],
                [4.4, 5.5, 6.6],
                [7.7, 8.8, 9.9]])
arr

In [ ]:
arr[:2, 1:]

<br>

In [ ]:
arr[2:, :]

<br>

In [ ]:
arr[:, :2]

<br>

In [ ]:
arr[1:2, :2]

<center>
<img src="../img/numpy-memory-layout.png" width="75%">
</center>

<center>
<img src="../img/numpy-memory-reshape.png" width="75%">
</center>

<center>
<img src="../img/numpy-memory-slice.png" width="75%">
</center>

**Quizlet:** Slice `three_dimensional` such that it looks like

```python
[[ 4  9]
 [24 29]]
```

In [ ]:
three_dimensional = np.arange(30).reshape((3, 2, 5))
three_dimensional

<br>

In [ ]:
three_dimensional[ : : ]

### Advanced slicing

Not only that, but NumPy can slice arrays with arrays.

In [ ]:
arr  = np.array([  0.0,   1.1,   2.2,   3.3,   4.4,  5.5,   6.6,  7.7,   8.8,  9.9])
mask = np.array([False, False, False, False, False, True, False, True, False, True])
#                                                    5.5          7.7          9.9

<br>

In [ ]:
arr[mask]

<br>

An array of integers picks out elements by index.

In [ ]:
arr[np.array([5, 7, -1])]

<br>

They can be out of order.

In [ ]:
arr[np.array([-1, 7, 5])]

<br>

They can even include duplicates.

In [ ]:
arr[np.array([-1, -1, -1, 7, 7, 5])]

In [ ]:
text = """
WOULD YOU LIKE GREEN EGGS AND HAM?

I DO NOT LIKE THEM, SAM-I-AM.
I DO NOT LIKE GREEN EGGS AND HAM.

WOULD YOU LIKE THEM HERE OR THERE?

I WOULD NOT LIKE THEM HERE OR THERE.
I WOULD NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU LIKE THEM IN A HOUSE?
WOULD YOU LIKE THEN WITH A MOUSE?

I DO NOT LIKE THEM IN A HOUSE.
I DO NOT LIKE THEM WITH A MOUSE.
I DO NOT LIKE THEM HERE OR THERE.
I DO NOT LIKE THEM ANYWHERE.
I DO NOT LIKE GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.

WOULD YOU EAT THEM IN A BOX?
WOULD YOU EAT THEM WITH A FOX?

NOT IN A BOX. NOT WITH A FOX.
NOT IN A HOUSE. NOT WITH A MOUSE.
I WOULD NOT EAT THEM HERE OR THERE.
I WOULD NOT EAT THEM ANYWHERE.
I WOULD NOT EAT GREEN EGGS AND HAM.
I DO NOT LIKE THEM, SAM-I-AM.
"""

In [ ]:
words = np.array(text.replace(",", " ").replace(".", " ").replace("?", " ").replace("!", " ").replace("-", " ").split())
dictionary, index = np.unique(words, return_inverse=True)

<br>

In [ ]:
dictionary

<br>

In [ ]:
index

**Quizlet:** What's going to happen?

<br>

In [ ]:
dictionary[index]

<br>

<details>
    <summary><b>Hint...</b></summary>

<br>

```
index             : positions in corpus (0, 1, 2, ...) → integer codes
dictionary        : integer codes                      → words

dictionary[index] : positions in corpus (0, 1, 2, ...) → words
```

</details>

### Reductions

We've seen operations that apply to each element of an array, producing a new array of the same length ("map").

NumPy also has operations that turn $n$-dimensional arrays into $(n-1)$-dimensional arrays ("reduce").

<center>
<img src="../img/example-reducer-2d.svg" width="50%">
</center>

In [ ]:
arr = np.array([[  1,   2,   3,   4],
                [ 10,  20,  30,  40],
                [100, 200, 300, 400]])

<br>

In [ ]:
np.sum(arr, axis=0)

<br>

In [ ]:
np.sum(arr, axis=1)

**Example:** Downsample this curve to make it less noisy.

<br>

In [ ]:
noisy_data = np.sin(np.linspace(0, 2*np.pi, 1000)) + np.random.normal(0, 0.5, 1000)

<br>

In [ ]:
plt.plot(noisy_data);

That is, replace every 10 consecutive array elements with their average value, reducing the number of array elements from 1000 to 100, but representing the same curve (ranging from ‒1 to 1).

<br>

In [ ]:
plt.plot(np.sum(noisy_data.reshape(-1, 10), axis=1));